In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '6'
from torch.utils.data import DataLoader
import sys
sys.path.append('/home/mei/nas/docker/thesis/model_train')
from dataloader.ts_reader import MultiModalDataset,VitalSignsDataset,vital_pre_train
from model.auencoder_v2 import *
from model.autoencoder_v2_loss_train import *
import seaborn as sns
import matplotlib.pyplot as plt
import gc
gc.collect()
import torch.optim as optim
import torch
torch.cuda.empty_cache()

In [2]:

vital_signs_train = '/home/mei/nas/docker/thesis/data/hdf/train/ts_each_patient.h5'
vital_signs_val = '/home/mei/nas/docker/thesis/data/hdf/val/ts_each_patient.h5'
vital_signs_test = '/home/mei/nas/docker/thesis/data/hdf/test/ts_each_patient.h5'

train_dataset = VitalSignsDataset(vital_signs_train)
val_dataset = VitalSignsDataset(vital_signs_val)
test_dataset = VitalSignsDataset(vital_signs_test)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=vital_pre_train)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, collate_fn=vital_pre_train) 
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, collate_fn=vital_pre_train)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model  = PatientAutoencoder(
    n_features=154, hidden_dim=128, latent_dim=100,
                 grid_size=(10,10), lstm_hidden_dim=128, n_heads=2
).to(device)

In [ ]:
base_dir = "/home/mei/nas/docker/thesis/data/model_results/autoencoder"
stage1_dir = os.path.join(base_dir, "stage1")
os.makedirs(stage1_dir, exist_ok=True)

pretrain_epochs  = 100
prior_max        = 1e-5
anneal_steps     = 200
opt_vae = optim.Adam(model.parameters(), lr=1e-3)

hist_pre = train_vae_pretrain(
    model,
    loader=train_loader,
    optimizer=opt_vae,
    device=device,
    checkpoint_dir=stage1_dir,
    epochs=pretrain_epochs,
    prior_max=prior_max,
    anneal_steps=anneal_steps
)

with open(os.path.join(stage1_dir, "history_pretrain.json"), "w") as f:
    json.dump(hist_pre, f, indent=2)
torch.save(
    model.state_dict(),
    os.path.join(stage1_dir, "best_model_vae.pth")
)

In [ ]:
stage2_dir = os.path.join(base_dir, "stage2")
os.makedirs(stage2_dir, exist_ok=True)

som_epochs_stage = 50//3
som_lrs=(0.1,0.01,0.001)
grid_size=(10,10)


hist_som = init_som_centroids(
    model,
    loader=train_loader,
    device=device,
    checkpoint_dir=stage2_dir,
    epochs_per_stage=som_epochs_stage,
    lrs=som_lrs,
    grid_size=grid_size
)

with open(os.path.join(stage2_dir, "history_som_init.json"), "w") as f:
    json.dump(hist_som, f, indent=2)
torch.save(
    model.state_dict(),
    os.path.join(stage2_dir, "best_model_som_init.pth")
)

In [ ]:
joint_epochs=100
base_lr=1e-3
decay=0.99
weights = {
        'recon': 1.0, 'kl': 1.0, 'cluster': 1.0,
        'ssom': 10.0, 'smooth': 1.0, 'pred': 1.0,
        'cluster_k': 2.0
    }


stage3_dir = os.path.join(base_dir, "stage3")
os.makedirs(stage3_dir, exist_ok=True)

hist_joint = joint_train(
    model,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    checkpoint_dir=stage3_dir,
    epochs=joint_epochs,
    base_lr=base_lr,
    decay=decay,
    weights=weights,
    grid_size=(10,10)
)

with open(os.path.join(stage3_dir, "history_joint_train.json"), "w") as f:
    json.dump(hist_joint, f, indent=2)
torch.save(
    model.state_dict(),
    os.path.join(stage3_dir, "best_model_joint.pth")
)

[Joint] Epoch 10/10  train=127.1459  val=122.9113
  ➡ saved /home/mei/nas/docker/thesis/data/model_results/autoencoder/stage3/joint_epoch010.pth


In [ ]:
pred_epochs = 50

stage4_dir = os.path.join(base_dir, "stage4")
os.makedirs(stage4_dir, exist_ok=True)

hist_pred = predict_finetune(
    model,
    loader=train_loader,
    device=device,
    checkpoint_dir=stage4_dir,
    epochs=pred_epochs
)

with open(os.path.join(stage4_dir, "history_pred_finetune.json"), "w") as f:
    json.dump(hist_pred, f, indent=2)
torch.save(
    model.state_dict(),
    os.path.join(stage4_dir, "best_model_pred.pth")
)

[Pred Finetune] ep 5/5  loss=0.4412
  ➡ saved /home/mei/nas/docker/thesis/data/model_results/autoencoder/stage4/predfinetune_epoch005.pth


In [ ]:
import matplotlib.pyplot as plt

def plot_stage_losses(hist_pre, hist_som, hist_joint, hist_pred):
    """
    绘制四个训练阶段的 loss 曲线：
      - hist_pre   : VAE 预训练阶段 loss list
      - hist_som   : SOM centroid 初始化阶段 loss list
      - hist_joint : 联合训练阶段 loss list
      - hist_pred  : 预测微调阶段 loss list
    """
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    
    # Stage 1: VAE Pretrain
    ax = axes[0, 0]
    ax.plot(range(1, len(hist_pre) + 1), hist_pre, marker='o')
    ax.set_title("Stage1: VAE Pretrain")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss")
    ax.grid(True)
    
    # Stage 2: SOM Init
    ax = axes[0, 1]
    ax.plot(range(1, len(hist_som) + 1), hist_som, color='orange', marker='o')
    ax.set_title("Stage2: SOM Init Centroids")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss")
    ax.grid(True)
    
    # Stage 3: Joint Train
    ax = axes[1, 0]
    ax.plot(range(1, len(hist_joint["train"]) + 1), hist_joint["train"], label='Train')
    ax.plot(range(1, len(hist_joint["val"]  ) + 1), hist_joint["val"],   label='Val', linestyle='--')
    ax.set_title("Stage3: Joint Train")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss")
    ax.legend()
    ax.grid(True)
    
    # Stage 4: Prediction Finetune
    ax = axes[1, 1]
    ax.plot(range(1, len(hist_pred) + 1), hist_pred, color='green', marker='o')
    ax.set_title("Stage4: Predict Finetune")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss")
    ax.grid(True)
    
    plt.tight_layout()
    plt.show()


In [ ]:
plot_stage_losses(hist_pre, hist_som, hist_joint, hist_pred)

In [ ]:
with open('/home/mei/nas/docker/thesis/data/hdf/features.txt', 'r') as f:
    features = [line.strip() for line in f]

selected_feature_indices = [0,1,2,142,143,144,145,146]  #

visualize_recons(model, test_loader, num_patients=5, feature_indices=selected_feature_indices, feature_names=features,device=device)